In [4]:
import pandas as pd
import numpy as np
import pickle as pkl
import os
import virtual_biopsy_utils as vbu
import integration_images_features_utils as image_utils
import ast

### Load sentara

In [5]:
sen_data = vbu.load_sentara(path = '../pkls/sentara.pkl', overwrite=True)

shape: (2395, 7714)


In [15]:
h = pd.read_csv('../input_files/tal_predictions_heldout.csv')


In [16]:
v = pd.read_csv('../input_files/tal_predictions_val.csv')

In [17]:
t = pd.read_csv('../input_files/tal_predictions_train.csv')

In [28]:
frames = [t, v, h]
data = pd.concat(frames)
data = data[data['provider'] == 'sentara']
data = data[data['viewpoint'] != 'LMO']
data = data[data['viewpoint'] != 'MLOIMF']
data.groupby('viewpoint').size().sort_values(ascending=False)

viewpoint
MLO     2552
CC      2399
ML         4
SCC        2
MCC        1
MML        1
SMLO       1
XCCL       1
dtype: int64

In [6]:
pred_file_path = '../input_files/final_tal_predictons_without_annotation_all.csv'
pred= pd.read_csv(pred_file_path)

train = [x.split('\t') for x in open('../input_files/sentara_train_studies.txt').readlines()]
val = [x.split('\t') for x in open('../input_files/sentara_val_studies.txt').readlines()]
test = [x.split('\t') for x in open('../input_files/sentara_test_studies.txt').readlines()]

studies_train = [item[1] for item in train[1:]]
studies_val = [item[1] for item in val[1:]]
studies_test = [item[1] for item in test[1:]]

pats_train = [item[0] for item in train[1:]]
pats_val = [item[0] for item in val[1:]]
pats_test = [item[0] for item in test[1:]]

FileNotFoundError: [Errno 2] No such file or directory: '../input_files/final_tal_predictons_without_annotation_all.csv'

In [103]:
images = pred.image_id.unique()

lesions_per_image = []
for i in images:
    lesions_per_image.append(pred[pred['image_id'] == i].finding_id.shape[0])

In [104]:
set(lesions_per_image)

{1}

In [109]:
pred.shape

(5819, 33)

In [108]:
len(list(pred.xml_url.unique()))

5819

In [74]:
len(list(set(pats_test)))

344

In [73]:
len(list(set(studies_test)))

353

In [75]:
len(list(set(x_test.index.get_level_values('patient_id').tolist())))

344

In [95]:
# number of images
pred[pred['study_id'].isin(studies_val)].shape[0]

722

In [96]:
len(list(pred[pred['study_id'].isin(studies_val)].xml_url.unique()))

722

In [42]:
imperst = []
for p in list(studies_train) + list(studies_val)+list(studies_test):
    imperst.append(pred[pred['study_id'] == p].shape[0])
set(imperst)

{0, 1, 2, 3, 4, 5, 6, 7}

In [20]:
len(list(set(sen_data.index.get_level_values('patient_id').tolist())))

2336

In [24]:
pats = list(set(sen_data.index.get_level_values('patient_id').tolist()))

In [25]:
sen_data[np.in1d(sen_data.index.get_level_values(1), pats[0])]

,,,study_id,outcome_biopsy_result_1yr,outcome_bc_1yr,outcome_bc_2yr,outcome_calc_max_birads,outcome_birads_1yr,prev_biopsy_result_max,prev_birads_ind,breast_MRI_cnt,breast_MRI_ind,...,bmi_variance,bmi_change,months_since_first_MG,race,religion,outcome_cancer_type_DCIS,outcome_cancer_type_Invasive,outcome_cancer_type_BenignHR,outcome_cancer_type_Papilloma,outcome_cancer_type_Benign
,patient_id,study_date,,,,,,,,,,,,,,,,,,,,,
1490,50700289,2013-08-22,MG130822004599,2.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,1.0,1.0,0,0,0,1,0


In [26]:
n_studies = []
for p in pats:
    n_studies.append(sen_data[np.in1d(sen_data.index.get_level_values(1), p)].shape[0])
    

In [28]:
set(n_studies)

{1, 2, 3}

In [18]:
max(list(set(sen_data.index.get_level_values('study_date').tolist())))

Timestamp('2018-11-01 00:00:00')

### Load shared features between maccabi and sentara 

In [21]:
x_train_path = '../pkls/x_train_ready_for_training.pkl'
x_train_after = pkl.load(open(x_train_path, 'rb'))

In [22]:
shared = pkl.load(open('../pkls/shared_features_mac_sen.pkl', 'rb'))

len(shared)

1842

### Add calculated features to sentara

* Add BMI estimation

In [ ]:
sen_data = vbu.add_bmi_estimation_sentara(df=sen_data)

* Add likelihood of obesity estimation

In [ ]:
sen_data = vbu.add_likelihood_obesity_estimation_sentara(df=sen_data)

* Breast density estimation

In [ ]:
sen_data = vbu.add_density_estimation_sentara(df = sen_data)

### Use shared  features only

In [ ]:
sen_data = sen_data[shared]

### Split data

In [60]:
x_train, y_train, x_val, y_val, x_test, y_test = vbu.split_sentara(sen_data, 
                                train_path = '../pkls/sentara_train.pkl',
                                val_path = '../pkls/sentara_val.pkl', 
                                test_path = '../pkls/sentara_test.pkl', overwrite = True)

Number of samples in train: 1685, val: 357 and test: 353


### Transform multilabel to single label - only on the train set

In [ ]:
sen_data_train = x_train.combine_first(y_train)

cancers = [x for x in sen_data_train if 'outcome_cancer_type' in x]

temp = []
for _, row in sen_data_train.iterrows():
    
    cols_one = row[cancers].loc[lambda x:x==True].index
    for col_name in cols_one:
        temp_row = row.copy()
        temp_row[cancers] = 0
        temp_row[col_name] = 1
        temp.append(temp_row)
        
sen_data_train = pd.DataFrame(temp)
multiindex = sen_data_train.index.tolist()
sen_data_train.reset_index(inplace=True)
sen_data_train['patient_id'] = [item[1] for item in multiindex]
sen_data_train['study_date'] = [item[2] for item in multiindex]
sen_data_train.set_index(['patient_id', 'study_date'], inplace = True, append=True)
sen_data_train.drop('index', axis = 1, inplace=True)
print('New number of samples of training set: %d' %sen_data_train.shape[0])

# Redefine new training dataframes with the single label samples
y_train = sen_data_train[[x for x in sen_data_train.columns if x.startswith('outcome_cancer_')]]
x_train = sen_data_train.drop(columns=[x for x in sen_data_train.columns if x.startswith('outcome_')])

### Add imaging features to all sets

In [ ]:
# file from image classifier with entire set (train, test, val)
pred_file_path = '../input_files/final_tal_predictons_without_annotation_all.csv'

* Predictions

In [ ]:
pred = image_utils.compute_predictions_images_sentara(pred_file_path)

x_train = x_train.join(pred.set_index('study_id'), on='study_id')
x_val = x_val.join(pred.set_index('study_id'), on='study_id')
x_test = x_test.join(pred.set_index('study_id'), on='study_id')

* Findings size

In [ ]:
# finding_size = image_utils.compute_findings_size(pred_file_path)

# x_train = x_train.join(finding_size.set_index('study_id'), on='study_id')
# x_val = x_val.join(finding_size.set_index('study_id'), on='study_id')
# x_test = x_test.join(finding_size.set_index('study_id'), on='study_id')

* Types: Calcification, breast assymetry, tumor, architectural distortion, axillary lymphadenopa

In [ ]:
# types = image_utils.get_types(pred_file_path)

# x_train = x_train.join(types.set_index('study_id'), on='study_id')
# x_val = x_val.join(types.set_index('study_id'), on='study_id')
# x_test = x_test.join(types.set_index('study_id'), on='study_id')

* Microcalcifications and Macrocalcifications

In [ ]:
# micro = image_utils.get_micro_and_macro(pred_file_path)

# x_train = x_train.join(micro.set_index('study_id'), on='study_id')
# x_val = x_val.join(micro.set_index('study_id'), on='study_id')
# x_test = x_test.join(micro.set_index('study_id'), on='study_id')

* Ella's features

In [ ]:
# spic_lesions_studies = [x.split('\t') for x in open('../input_files/additional_features_spiculated_lesions_sentara.txt').readlines()]
# arch_dist_studies = [x.split('\t') for x in open('../input_files/additional_features_architectural_distortion_sentara.txt').readlines()]
# susp_calc_studies = [x.split('\t') for x in open('../input_files/additional_features_suspicius_calcifications_sentara.txt').readlines()]

# spic_lesions_studies = [item[1][:-1] for item in spic_lesions_studies[1:]]
# arch_dist_studies = [item[1][:-1] for item in arch_dist_studies[1:]]
# susp_calc_studies = [item[1][:-1] for item in susp_calc_studies[1:]]

# x_train['spiculated_lesions_report'] = np.array([x in spic_lesions_studies for x in x_train.study_id.tolist()]).astype(int)
# x_train['architectural_distortion_report'] = np.array([x in arch_dist_studies for x in x_train.study_id.tolist()]).astype(int)
# x_train['suspicious_calcifications_report'] = np.array([x in susp_calc_studies for x in x_train.study_id.tolist()]).astype(int)

# x_val['spiculated_lesions_report'] = np.array([x in spic_lesions_studies for x in x_val.study_id.tolist()]).astype(int)
# x_val['architectural_distortion_report'] = np.array([x in arch_dist_studies for x in x_val.study_id.tolist()]).astype(int)
# x_val['suspicious_calcifications_report'] = np.array([x in susp_calc_studies for x in x_val.study_id.tolist()]).astype(int)


# x_test['spiculated_lesions_report'] = np.array([x in spic_lesions_studies for x in x_test.study_id.tolist()]).astype(int)
# x_test['architectural_distortion_report'] = np.array([x in arch_dist_studies for x in x_test.study_id.tolist()]).astype(int)
# x_test['suspicious_calcifications_report'] = np.array([x in susp_calc_studies for x in x_test.study_id.tolist()]).astype(int)


In [ ]:
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape )

In [ ]:
with open('../pkls/x_test_ready_for_testing.pkl', 'wb') as handle:
    pkl.dump(x_test, handle, protocol=pkl.HIGHEST_PROTOCOL)

with open('../pkls/y_test_ready_for_testing.pkl', 'wb') as handle:
    pkl.dump(y_test, handle, protocol=pkl.HIGHEST_PROTOCOL)
    
with open('../pkls/x_train_ready_for_training.pkl', 'wb') as handle:
    pkl.dump(x_train, handle, protocol=pkl.HIGHEST_PROTOCOL)
    
with open('../pkls/y_train_ready_for_training.pkl', 'wb') as handle:
    pkl.dump(y_train, handle, protocol=pkl.HIGHEST_PROTOCOL)   
    
with open('../pkls/x_val_ready_for_training.pkl', 'wb') as handle:
    pkl.dump(x_val, handle, protocol=pkl.HIGHEST_PROTOCOL)
    
with open('../pkls/y_val_ready_for_training.pkl', 'wb') as handle:
    pkl.dump(y_val, handle, protocol=pkl.HIGHEST_PROTOCOL)